In [32]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate,ChatMessagePromptTemplate,SystemMessagePromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_community.vectorstores import VectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
load_dotenv()


True

In [33]:
from langchain_community.document_loaders import TextLoader

file_path = r"C:\Users\91800\Desktop\GENAI UDEMY\txt_files\gemini.txt"
try:
    loader = TextLoader(file_path, encoding="utf-8")
    data = loader.load()
    print(f"Loaded {len(data)} documents using utf-8")
except UnicodeDecodeError as e:
    print(f"utf-8 UnicodeDecodeError: {e}; trying latin-1 with replacement")
    loader = TextLoader(file_path, encoding="latin-1", errors="replace")
    data = loader.load()
    print(f"Loaded {len(data)} documents using latin-1 (with replacement)")
except Exception as e:
    print(f"Error loading file: {e}")

# preview
if isinstance(data, list):
    for i, doc in enumerate(data, 1):
        content = getattr(doc, "page_content", None) or getattr(doc, "content", None) or str(doc)
        print(f"--- Document {i} ---")
        print(content[:1000])
        print()
else:
    content = getattr(data, "page_content", None) or getattr(data, "content", None) or str(data)
    print(content[:1000])


Loaded 1 documents using utf-8
--- Document 1 ---
Mars is the fourth planet from the Sun and the second-smallest planet in the Solar System, being larger than only Mercury.
Mars is often called the 'Red Planet' because the iron oxide prevalent on its surface gives it a reddish appearance.
The atmosphere of Mars is very thin, composed mainly of carbon dioxide (95.32%), with smaller amounts of nitrogen and argon.
Mars has two small, irregularly shaped moons, Phobos and Deimos. They are thought to be captured asteroids.
A day on Mars (a 'sol') is slightly longer than an Earth day, lasting 24.6 hours. A Martian year is significantly longer, lasting 687 Earth days.
Mars has the largest volcano in the Solar System, Olympus Mons, and the largest canyon, Valles Marineris.
The first successful Mars flyby was NASA's Mariner 4 mission in 1965. The first rovers to land, Sojourner, Spirit, and Opportunity, arrived much later.
Liquid water cannot exist on the surface of Mars long-term due to its low

# After we have loaded the data we are going to split this data using the data splitter 


In [34]:
recursive_spliiter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=40)

In [35]:
documents=recursive_spliiter.split_documents(data)

In [36]:
len(documents)

3

# Now once we have splitted the data and all it is ready for the Embeddings

In [37]:
import os
from dotenv import load_dotenv
embedding_model=GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model2 = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [38]:
vectorStore = Chroma.from_documents(
    documents=documents, 
    embedding=embedding_model2
)

# Now we are going to store the embedding of the model into vector database for the fast retriveal 

In [39]:
from langchain_chroma import Chroma

In [40]:
retriever = vectorStore.as_retriever(search_kwargs={"k": 1})

In [41]:
retriever.invoke("Why is Mars called the Red Planet?")

[Document(id='fe8b0c70-2b8f-47d2-bace-d91d5c66dfdc', metadata={'source': 'C:\\Users\\91800\\Desktop\\GENAI UDEMY\\txt_files\\gemini.txt'}, page_content="Mars is the fourth planet from the Sun and the second-smallest planet in the Solar System, being larger than only Mercury.\nMars is often called the 'Red Planet' because the iron oxide prevalent on its surface gives it a reddish appearance.\nThe atmosphere of Mars is very thin, composed mainly of carbon dioxide (95.32%), with smaller amounts of nitrogen and argon.\nMars has two small, irregularly shaped moons, Phobos and Deimos. They are thought to be captured asteroids.")]

In [42]:
retrieved_docs = retriever.invoke("Why is Mars called the Red Planet?")
print(retrieved_docs)

[Document(id='fe8b0c70-2b8f-47d2-bace-d91d5c66dfdc', metadata={'source': 'C:\\Users\\91800\\Desktop\\GENAI UDEMY\\txt_files\\gemini.txt'}, page_content="Mars is the fourth planet from the Sun and the second-smallest planet in the Solar System, being larger than only Mercury.\nMars is often called the 'Red Planet' because the iron oxide prevalent on its surface gives it a reddish appearance.\nThe atmosphere of Mars is very thin, composed mainly of carbon dioxide (95.32%), with smaller amounts of nitrogen and argon.\nMars has two small, irregularly shaped moons, Phobos and Deimos. They are thought to be captured asteroids.")]
